In [10]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel
import joblib

# 1. 数据集类
class SoilDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=256):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        description = self.data.iloc[index]['description']
        encoding = self.tokenizer.encode_plus(
            description,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        spectral_data = self.data.iloc[index][['Centroid_X', 'Centroid_Y', 'DEM_MEAN', 'DEM_STD', 'ndvi_MEAN',
       'PCA_0_MEAN', 'Slope_MEAN', 'Aspect_MEAN', 'MRVBF_MEAN',
       'TopographicWetnessIndex_MEAN', 'Mean_MEAN', 'PH_MEAN', 'PRE_MEAN',
       'SRA_MEAN', 'TMP_MEAN', 'VAP_MEAN', 'WIND_MEAN', 'PlanCurvature_MEAN']].values.astype(float)

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'spectral_data': torch.FloatTensor(spectral_data),
            'soil_class': torch.tensor(self.data.iloc[index]['soil_class_encoded'], dtype=torch.long),
            'soil_subclass': torch.tensor(self.data.iloc[index]['soil_subclass_encoded'], dtype=torch.long),
            'soil_group': torch.tensor(self.data.iloc[index]['soil_group_encoded'], dtype=torch.long),
            'soil_type': torch.tensor(self.data.iloc[index]['soil_type_encoded'], dtype=torch.long)
        }

# 2. 模型定义
class RevisedHierarchicalSoilClassificationModel(nn.Module):
    def __init__(self, n_classes, n_spectral_features=18):
        super(RevisedHierarchicalSoilClassificationModel, self).__init__()
        self.n_classes = n_classes
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(0.1)
        self.spectral_fc = nn.Linear(n_spectral_features, 64)
        self.fc = nn.Linear(self.bert.config.hidden_size + 64, 256)
        
        # Revised hierarchical classification layers
        self.soil_class = nn.Linear(256, n_classes['soil_class'])
        self.soil_subclass = nn.Linear(256, n_classes['soil_class'] * n_classes['soil_subclass'])
        self.soil_group = nn.Linear(256, n_classes['soil_subclass'] * n_classes['soil_group'])
        self.soil_type = nn.Linear(256, n_classes['soil_group'] * n_classes['soil_type'])

    def forward(self, input_ids, attention_mask, spectral_data):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        pooled_output = self.dropout(pooled_output)
        
        spectral_features = self.spectral_fc(spectral_data)
        spectral_features = torch.relu(spectral_features)
        
        combined = torch.cat((pooled_output, spectral_features), dim=1)
        x = self.fc(combined)
        x = torch.relu(x)
        
        # Hierarchical prediction
        soil_class = self.soil_class(x)
        soil_subclass = self.soil_subclass(x).view(-1, self.n_classes['soil_class'], self.n_classes['soil_subclass'])
        soil_group = self.soil_group(x).view(-1, self.n_classes['soil_subclass'], self.n_classes['soil_group'])
        soil_type = self.soil_type(x).view(-1, self.n_classes['soil_group'], self.n_classes['soil_type'])
        
        return soil_class, soil_subclass, soil_group, soil_type

# 3. 训练函数
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            spectral_data = batch['spectral_data'].to(device)
            soil_class = batch['soil_class'].to(device)
            soil_subclass = batch['soil_subclass'].to(device)
            soil_group = batch['soil_group'].to(device)
            soil_type = batch['soil_type'].to(device)

            optimizer.zero_grad()
            class_out, subclass_out, group_out, type_out = model(input_ids, attention_mask, spectral_data)

            # 调试输出
            # print(f"input_ids: {input_ids.shape}, attention_mask: {attention_mask.shape}, spectral_data: {spectral_data.shape}")
            # print(f"soil_class: {soil_class}, soil_subclass: {soil_subclass}, soil_group: {soil_group}, soil_type: {soil_type}")
            
            # 确保索引在范围内
            assert soil_class.max().item() < model.n_classes['soil_class'], "soil_class index out of range"
            assert soil_subclass.max().item() < model.n_classes['soil_subclass'], "soil_subclass index out of range"
            assert soil_group.max().item() < model.n_classes['soil_group'], "soil_group index out of range"
            assert soil_type.max().item() < model.n_classes['soil_type'], "soil_type index out of range"

            # Calculate loss for each level
            loss_class = criterion(class_out, soil_class)
            loss_subclass = criterion(subclass_out[torch.arange(subclass_out.size(0)), soil_class], soil_subclass)
            loss_group = criterion(group_out[torch.arange(group_out.size(0)), soil_subclass], soil_group)
            loss_type = criterion(type_out[torch.arange(type_out.size(0)), soil_group], soil_type)
            
            loss = loss_class + loss_subclass + loss_group + loss_type
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                spectral_data = batch['spectral_data'].to(device)
                soil_class = batch['soil_class'].to(device)
                soil_subclass = batch['soil_subclass'].to(device)
                soil_group = batch['soil_group'].to(device)
                soil_type = batch['soil_type'].to(device)

                class_out, subclass_out, group_out, type_out = model(input_ids, attention_mask, spectral_data)
                
                # Calculate loss for each level
                loss_class = criterion(class_out, soil_class)
                loss_subclass = criterion(subclass_out[torch.arange(subclass_out.size(0)), soil_class], soil_subclass)
                loss_group = criterion(group_out[torch.arange(group_out.size(0)), soil_subclass], soil_group)
                loss_type = criterion(type_out[torch.arange(type_out.size(0)), soil_group], soil_type)
                
                loss = loss_class + loss_subclass + loss_group + loss_type
                val_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

# 4. 评估函数
def evaluate_model(model, data_loader, device):
    model.eval()
    correct_predictions = {
        'soil_class': 0,
        'soil_subclass': 0,
        'soil_group': 0,
        'soil_type': 0
    }
    total_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            spectral_data = batch['spectral_data'].to(device)
            soil_class = batch['soil_class'].to(device)
            soil_subclass = batch['soil_subclass'].to(device)
            soil_group = batch['soil_group'].to(device)
            soil_type = batch['soil_type'].to(device)

            class_out, subclass_out, group_out, type_out = model(input_ids, attention_mask, spectral_data)
            
            _, class_preds = torch.max(class_out, dim=1)
            _, subclass_preds = torch.max(subclass_out[torch.arange(subclass_out.size(0)), class_preds], dim=1)
            _, group_preds = torch.max(group_out[torch.arange(group_out.size(0)), subclass_preds], dim=1)
            _, type_preds = torch.max(type_out[torch.arange(type_out.size(0)), group_preds], dim=1)
            
            correct_predictions['soil_class'] += torch.sum(class_preds == soil_class)
            correct_predictions['soil_subclass'] += torch.sum(subclass_preds == soil_subclass)
            correct_predictions['soil_group'] += torch.sum(group_preds == soil_group)
            correct_predictions['soil_type'] += torch.sum(type_preds == soil_type)
            
            total_predictions += input_ids.size(0)

    accuracies = {k: (v.item() / total_predictions) * 100 for k, v in correct_predictions.items()}
    return accuracies

# 5. 预测类
class RevisedConstrainedHierarchicalSoilClassifier:
    def __init__(self, model_path, tokenizer_path, label_encoders, hierarchy_map, device=None):
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.model = torch.load(model_path, map_location=self.device)
        self.model.eval()
        self.label_encoders = label_encoders
        self.hierarchy_map = hierarchy_map

    def predict(self, text, spectral_data):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        spectral_tensor = torch.FloatTensor(spectral_data).unsqueeze(0).to(self.device)

        with torch.no_grad():
            class_out, subclass_out, group_out, type_out = self.model(input_ids, attention_mask, spectral_tensor)
        
        try:
            # Constrained prediction
            soil_class_idx = torch.argmax(class_out, dim=1).item()
            soil_class = self.label_encoders['soil_class'].inverse_transform([soil_class_idx])[0]
            
            # print(f"Predicted soil_class: {soil_class}")
            # print(f"Valid subclasses for {soil_class}: {list(self.hierarchy_map[soil_class].keys())}")
            
            subclass_probs = subclass_out[0, soil_class_idx]
            valid_subclass_idxs = [self.label_encoders['soil_subclass'].transform([subclass])[0] for subclass in self.hierarchy_map[soil_class].keys()]
            valid_subclass_probs = subclass_probs[valid_subclass_idxs]
            soil_subclass_idx = valid_subclass_idxs[torch.argmax(valid_subclass_probs)]
            soil_subclass = self.label_encoders['soil_subclass'].inverse_transform([soil_subclass_idx])[0]
            
            # print(f"Predicted soil_subclass: {soil_subclass}")
            # print(f"Valid groups for {soil_subclass}: {list(self.hierarchy_map[soil_class][soil_subclass].keys())}")
            
            group_probs = group_out[0, soil_subclass_idx]
            valid_group_idxs = [self.label_encoders['soil_group'].transform([group])[0] for group in self.hierarchy_map[soil_class][soil_subclass].keys()]
            valid_group_probs = group_probs[valid_group_idxs]
            soil_group_idx = valid_group_idxs[torch.argmax(valid_group_probs)]
            soil_group = self.label_encoders['soil_group'].inverse_transform([soil_group_idx])[0]
            
            # print(f"Predicted soil_group: {soil_group}")
            # print(f"Valid types for {soil_group}: {self.hierarchy_map[soil_class][soil_subclass][soil_group]}")
            
            type_probs = type_out[0, soil_group_idx]
            valid_type_idxs = [self.label_encoders['soil_type'].transform([type_])[0] for type_ in self.hierarchy_map[soil_class][soil_subclass][soil_group]]
            valid_type_probs = type_probs[valid_type_idxs]
            soil_type_idx = valid_type_idxs[torch.argmax(valid_type_probs)]
            soil_type = self.label_encoders['soil_type'].inverse_transform([soil_type_idx])[0]
            
            results = {
                'soil_class': soil_class,
                'soil_subclass': soil_subclass,
                'soil_group': soil_group,
                'soil_type': soil_type
            }
            
            return results
        except Exception as e:
            print(f"Error in prediction: {str(e)}")
            print(f"soil_class_idx: {soil_class_idx}")
            print(f"soil_class: {soil_class}")
            print(f"subclass_out shape: {subclass_out.shape}")
            print(f"group_out shape: {group_out.shape}")
            print(f"type_out shape: {type_out.shape}")
            raise

# 6. 创建层级映射函数
def create_hierarchy_map(data):
    hierarchy_map = {}
    for _, row in data.iterrows():
        soil_class = row['soil_class']
        soil_subclass = row['soil_subclass']
        soil_group = row['soil_group']
        soil_type = row['soil_type']
        
        if soil_class not in hierarchy_map:
            hierarchy_map[soil_class] = {}
        if soil_subclass not in hierarchy_map[soil_class]:
            hierarchy_map[soil_class][soil_subclass] = {}
        if soil_group not in hierarchy_map[soil_class][soil_subclass]:
            hierarchy_map[soil_class][soil_subclass][soil_group] = []
        if soil_type not in hierarchy_map[soil_class][soil_subclass][soil_group]:
            hierarchy_map[soil_class][soil_subclass][soil_group].append(soil_type)
    
    return hierarchy_map

# 7. 主函数
def main(data, num_epochs, lr=2e-5):
    # 模型和标签存放位置
    save_path = 'model_path'
    os.makedirs(save_path, exist_ok=True)

    # 生成数据
    data = data.copy()
    
    # 编码标签
    label_encoders = {}
    for column in ['soil_class', 'soil_subclass', 'soil_group', 'soil_type']:
        le = LabelEncoder()
        data[f'{column}_encoded'] = le.fit_transform(data[column])
        label_encoders[column] = le
    
    # 创建层级映射
    hierarchy_map = create_hierarchy_map(data)
    
    # 划分训练集和测试集
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    
    # 初始化tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    
    # 创建数据集和数据加载器
    train_dataset = SoilDataset(train_data, tokenizer)
    test_dataset = SoilDataset(test_data, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16)
    
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 初始化模型
    n_classes = {
        'soil_class': len(label_encoders['soil_class'].classes_),
        'soil_subclass': len(label_encoders['soil_subclass'].classes_),
        'soil_group': len(label_encoders['soil_group'].classes_),
        'soil_type': len(label_encoders['soil_type'].classes_)
    }
    model = RevisedHierarchicalSoilClassificationModel(n_classes).to(device)
    
    # 设置损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # 训练模型
    train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)
    
    # 评估模型
    print("\n评估模型性能...")
    accuracies = evaluate_model(model, test_loader, device)
    
    print("模型准确率：")
    for category, accuracy in accuracies.items():
        print(f"{category}: {accuracy:.2f}%")
    
    # 保存模型
    torch.save(model, os.path.join(save_path, 'soil_classification_model.pth'))
    
    # 保存标签编码器
    joblib.dump(label_encoders, os.path.join(save_path, 'label_encoders.joblib'))
    
    # 保存层级映射
    joblib.dump(hierarchy_map, os.path.join(save_path, 'hierarchy_map.joblib'))
    
    # 保存tokenizer
    tokenizer.save_pretrained(os.path.join(save_path, 'soil_tokenizer'))

    print("模型训练完成并保存。")

# 8. 使用示例
def usage_example(sample_text, spectral_data):
    save_path = 'model_path'
    
    # 加载保存的模型和标签编码器
    label_encoders = joblib.load(os.path.join(save_path, 'label_encoders.joblib'))
    hierarchy_map = joblib.load(os.path.join(save_path, 'hierarchy_map.joblib'))

    classifier = RevisedConstrainedHierarchicalSoilClassifier(
    os.path.join(save_path, 'soil_classification_model.pth'),
    os.path.join(save_path, 'soil_tokenizer'),
    label_encoders,
    hierarchy_map
    )

    # 进行预测
    results = classifier.predict(sample_text, spectral_data)

    # 打印结果
    print("预测结果：")
    for key, value in results.items():
        print(f"{key}: {value}")

# 调用主函数
# main(data, num_epochs=5)
# 使用示例
# usage_example("sample text", [0.1, 0.2, 0.3, ...])


In [6]:
data = pd.read_csv("result.csv")
data.rename(columns={'TL':'soil_class','YL':'soil_subclass','TS':'soil_group','TZ':'soil_type'},inplace=True)

In [30]:
for i in range(4024,4030,1):
    row_index = i
    test_text = data['description'][row_index]
    columns_from_fifth = data.iloc[:, 5:].iloc[row_index].tolist()
    result = data.iloc[:,1:5].iloc[row_index].to_list()
    print(test_text,type(test_text))
    print(result)
    usage_example(test_text,columns_from_fifth)
    print('-'*100)

地块成土母质为泥(页)岩,土地利用类型为旱地 <class 'str'>
['黄壤', '典型黄壤', '泥质黄壤', '厚层泥质黄壤']
预测结果：
soil_class: 黄壤
soil_subclass: 典型黄壤
soil_group: 泥砂质黄壤
soil_type: 腐中层泥砂质黄壤
----------------------------------------------------------------------------------------------------
地块成土母质为泥(页)岩,土地利用类型为旱地 <class 'str'>
['黄壤', '典型黄壤', '泥质黄壤', '中层泥质黄壤']
预测结果：
soil_class: 黄壤
soil_subclass: 典型黄壤
soil_group: 泥砂质黄壤
soil_type: 腐中层泥砂质黄壤
----------------------------------------------------------------------------------------------------
地块成土母质为泥(页)岩,土地利用类型为旱地 <class 'str'>
['黄壤', '典型黄壤', '泥质黄壤', '厚层泥质黄壤']
预测结果：
soil_class: 黄壤
soil_subclass: 典型黄壤
soil_group: 泥砂质黄壤
soil_type: 腐中层泥砂质黄壤
----------------------------------------------------------------------------------------------------
地块成土母质为泥(页)岩,土地利用类型为旱地 <class 'str'>
['黄壤', '典型黄壤', '泥质黄壤', '厚层泥质黄壤']
预测结果：
soil_class: 黄壤
soil_subclass: 典型黄壤
soil_group: 泥砂质黄壤
soil_type: 腐中层泥砂质黄壤
----------------------------------------------------------------------------------------------------


In [25]:
# 运行主程序
main(data, num_epochs=5, lr=2e-5)

Epoch 1/5, Train Loss: 34278.0603, Val Loss: 13817.1145
Epoch 2/5, Train Loss: 8654.2538, Val Loss: 5060.5778
Epoch 3/5, Train Loss: 3790.1478, Val Loss: 2998.6457
Epoch 4/5, Train Loss: 2391.2860, Val Loss: 1893.1288
Epoch 5/5, Train Loss: 1468.4637, Val Loss: 1179.8723

评估模型性能...
模型准确率：
soil_class: 71.46%
soil_subclass: 54.07%
soil_group: 15.27%
soil_type: 14.05%
模型训练完成并保存。
